In [1]:
import dask.dataframe as dd
import datetime as dt
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from pathlib import Path
import gensim

/home/chloezh/miniconda3/envs/pytorch_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mimiciv_path = '/nfs/turbo/med-kayvan-lab/Projects/HeartFailure/Data/Raw/physionet.org/files/'

In [3]:
tbl = pd.read_csv('mimic_cxr_note_chexpert.csv')

In [5]:
tbl.columns 

Index(['dicom_id', 'subject_id', 'study_id',
       'PerformedProcedureStepDescription', 'ViewPosition', 'Rows', 'Columns',
       'StudyDate', 'StudyTime', 'ProcedureCodeSequence_CodeMeaning',
       'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'note', 'Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices'],
      dtype='object')

In [10]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModel.from_pretrained("medicalai/ClinicalBERT")

Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
tbl['note'] = tbl.apply(lambda x: x['note'].replace('\n', ' '), axis=1)
tbl['note'] = tbl.apply(lambda x: x['note'].replace('\\', ' ').strip(), axis=1)

In [9]:
tbl['note'].iloc[0]

'FINAL REPORT  EXAMINATION:  CHEST (PA AND LAT)    INDICATION:  ___F with new onset ascites  // eval for infection    TECHNIQUE:  Chest PA and lateral    COMPARISON:  None.    FINDINGS:     There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral  nodular opacities that most likely represent nipple shadows. The  cardiomediastinal silhouette is normal.  Clips project over the left lung,  potentially within the breast. The imaged upper abdomen is unremarkable.  Chronic deformity of the posterior left sixth and seventh ribs are noted.    IMPRESSION:     No acute cardiopulmonary process.'

In [24]:
pt_batch = tokenizer(
    list(tbl['note'][:3]),
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
) # 768 dim; 147: num of tokens

In [47]:
pt_outputs = model(**pt_batch)

In [32]:
pt_outputs[0][0].shape

torch.Size([147, 768])

In [51]:
t = 'this is a test string'
label = 0
toks = tokenizer(
    t,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt",
)
tokenizer.decode(toks.get('input_ids')[0])
# [tokenizer.decode(i) for i in pt_batch.get('input_ids')]

'[CLS] this is a test string [SEP]'

In [ ]:
### 

In [56]:
h = model(**toks)

In [ ]:
h[0][0]